## Compare results for multiple results

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
from modules_img_analysis import *

In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

In [5]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_plot_spectrum_3d(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1


def f_compute_chisqr(dict_val,dict_sample,img_size):
    '''
    Compute chi-square values for sample w.r.t input images
    Input: 2 dictionaries with 4 keys for histogram and spectrum values and errors
    '''
    ### !!Both pixel histograms MUST have same bins and normalization!
    ### Compute chi-sqr
    ### Used in keras code : np.sum(np.divide(np.power(valhist - samphist, 2.0), valhist))
    ###  chi_sqr :: sum((Obs-Val)^2/(Val))
    
    chisqr_dict={}
    
    try: 
        val_dr=dict_val['hist_val'].copy()
        val_dr[val_dr<=0.]=1.0    ### Avoiding division by zero for zero bins

        sq_diff=(dict_val['hist_val']-dict_sample['hist_val'])**2

        size=len(dict_val['hist_val'])
        l1,l2=int(size*0.3),int(size*0.7)
        keys=['chi_1a','chi_1b','chi_1c','chi_1']
        
        for (key,start,end) in zip(keys,[0,l1,l2,0],[l1,l2,None,None]):  # 4 lists : small, medium, large pixel values and full 
            chisqr_dict.update({key:np.sum(np.divide(sq_diff[start:end],val_dr[start:end]))})

        idx=None  # Choosing the number of histograms to use. Eg : -5 to skip last 5 bins
    #     chisqr_dict.update({'chi_sqr1':})

        chisqr_dict.update({'chi_2':np.sum(np.divide(sq_diff[:idx],1.0))}) ## chi-sqr without denominator division
        chisqr_dict.update({'chi_imgvar':np.sum(dict_sample['hist_err'][:idx])/np.sum(dict_val['hist_err'][:idx])}) ## measures total spread in histograms wrt to input data

        idx=img_size
        spec_diff=(dict_val['spec_val']-dict_sample['spec_val'])**2
        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec1':np.sum(spec_diff[:idx]/dict_sample['spec_val'][:idx]**2)})

        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec2':np.sum(spec_diff[:idx]/dict_sample['spec_sdev'][:idx]**2)})
        
        spec_loss=1.0*np.log(np.mean((dict_val['spec_val'][:idx]-dict_sample['spec_val'][:idx])**2))+1.0*np.log(np.mean((dict_val['spec_sdev'][:idx]-dict_sample['spec_sdev'][:idx])**2))
        chisqr_dict.update({'chi_spec3':spec_loss})
    
    except Exception as e: 
        print(e)
        
        keys=['chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']
        chisqr_dict=dict.fromkeys(keys,np.nan)
        pass
    
    return chisqr_dict
    

In [6]:
# img_size=64
img_size=128

In [7]:
val_data_dict={'64':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing',
              '128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube'}

In [8]:
### Read validation data
# bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,100.5,5),np.arange(100.5,300.5,20),np.arange(300.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1)

bkgnd_dict={}
num_bkgnd=100
sigma_lst=[0.5,0.65,0.8,1.1]
labels_lst=[0,1,2,3]
### Extract validation data   
for label in labels_lst:
    fname=val_data_dict[str(img_size)]+'/norm_1_sig_{0}_train_val.npy'.format(sigma_lst[label])
    print(fname)
    samples=np.load(fname,mmap_mode='r')[-num_bkgnd:][:,0,:,:,:]
    
    dict_val=f_compute_hist_spect(samples,bins)
    bkgnd_dict[str(sigma_lst[label])]=dict_val
# del samples


/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_0.5_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_0.65_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_0.8_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_1.1_train_val.npy


In [9]:
df_runs=pd.DataFrame([])

In [10]:
### bkgnd: Get bkgnd chi-squares for different sigma wrt one standard sigma 
lst=[0.5,0.65,0.8,1.1]
param_label=0.65
for count,run in enumerate(lst):
    num=100
    fname=val_data_dict[str(img_size)]+'/norm_1_sig_{0}_train_val.npy'.format(run)
    print(fname)    
    samples=np.load(fname,mmap_mode='r')[1000:1000+num][:,0,:,:,:]
    dict1={'label':'ref_%s'%(run),'fname':''}
    dict1['num_imgs']=num
    dict1['sigma']=run
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(samples,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(bkgnd_dict[str(param_label)],dict_sample,img_size)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    
    df_runs=df_runs.append(dict1,ignore_index=True)


/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_0.5_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_0.65_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_0.8_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_1.1_train_val.npy


### Add pytorch runs

In [11]:
## Pytorch code
### Load images

parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/'


lst=[
#         ['20210628_73858_cgan_128_nodes1_lr0.0002_fm0_goodrun/',24970],
#      ['20210703_50233_cgan_128_nodes1_lr0.0002_vary_fm0_goodrun/',19360],
#      ['20210723_72050_cgan_128_nodes1_lr0.00002_finetune/',5070],
     ['20210723_74424_cgan_128_nodes1_lr0.000002_finetune/',1950],
#      ['20210726_173009_cgan_128_nodes1_lr0.000002_finetune/',3010],
    ]


# Prediction images 

for param_label in sigma_lst:
    for count,run in enumerate(lst):
        fldr,step=run[0],run[1]
#         print(count,run)
        prefix='run_%s_%s_%s'%(count,step,param_label)
        prefix='best_%s'%(param_label)
        key=prefix
        main_dir=parent_dir+fldr
        print(key)
        fname=glob.glob(main_dir+'images/'+'inference_*label-{0}_*_step-{1}.npy'.format(param_label,step))[0]
        dict1={'label':key,'fname':fname}
        images=np.load(dict1['fname'])     ### No transform needed here

        images=images[:,0,:,:,:]
        print(images.shape)
        print(np.max(images),np.min(images))

        dict1['num_imgs']=images.shape[0]
        dict1['sigma']=param_label
        print(dict1)
        ### Compute spectrum and histograms
        dict_sample=f_compute_hist_spect(images,bins)
        ### Compute chi squares
        dict_chisqrs=f_compute_chisqr(bkgnd_dict[str(param_label)],dict_sample,img_size)

        dict1.update(dict_sample)
        dict1.update(dict_chisqrs)
        del(images)
        df_runs=df_runs.append(dict1,ignore_index=True)

best_0.5
(32, 128, 128, 128)
0.99719584 -0.997398
{'label': 'best_0.5', 'fname': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210723_74424_cgan_128_nodes1_lr0.000002_finetune/images/inference_label-0.5_epoch-8_step-1950.npy', 'num_imgs': 32, 'sigma': 0.5}
best_0.65
(32, 128, 128, 128)
0.9983237 -0.9977228
{'label': 'best_0.65', 'fname': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210723_74424_cgan_128_nodes1_lr0.000002_finetune/images/inference_label-0.65_epoch-8_step-1950.npy', 'num_imgs': 32, 'sigma': 0.65}
best_0.8
(32, 128, 128, 128)
0.99810666 -0.9989379
{'label': 'best_0.8', 'fname': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210723_74424_cgan_128_nodes1_lr0.000002_finetune/images/inference_label-0.8_epoch-8_step-1950.npy', 'num_imgs': 32, 'sigma': 0.8}
best_1.1
(32, 128, 128, 128)
0.9995669 -0.99938416
{'label': 'best_1.1', '

In [12]:
df_runs.columns

Index(['chi_1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2', 'chi_imgvar',
       'chi_spec1', 'chi_spec2', 'chi_spec3', 'fname', 'hist_bin_centers',
       'hist_err', 'hist_val', 'label', 'num_imgs', 'sigma', 'spec_sdev',
       'spec_val'],
      dtype='object')

In [13]:
col_list=['label','sigma', 'chi_1', 'chi_1a', 'chi_spec1', 'chi_spec3','chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2', 'hist_bin_centers', 'hist_err', 'hist_val','spec_sdev', 'spec_val','num_imgs','fname']

df_runs=df_runs.reindex(columns=col_list)


In [14]:
df_runs

,label,sigma,chi_1,chi_1a,chi_spec1,chi_spec3,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,hist_bin_centers,hist_err,hist_val,spec_sdev,spec_val,num_imgs,fname
0,ref_0.5,0.50,0.084855,0.057658,1.027117,58.889042,0.026169,0.001028,0.030951,0.809446,567.714923,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.0026209318746665865, 0.0012307953312302102,...","[0.8475708290444763, 0.590315426330211, 0.0400...","[33061027.02429947, 9522816.482167501, 6243423...","[150054862.58426616, 82111263.64840499, 499411...",100.0,
1,ref_0.65,0.65,0.000219,0.000076,0.005281,49.566061,0.000088,0.000055,0.000021,1.006303,1.631127,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.0027108935448769697, 0.0012516292039354302,...","[1.003411341364514, 0.5017591247274171, 0.0444...","[46197623.65859872, 13045425.547887238, 707084...","[182684482.71750823, 99577321.95103155, 590998...",100.0,
2,ref_0.8,0.80,0.067837,0.031967,0.447982,56.679259,0.029915,0.005955,0.019054,1.404328,431.451944,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.0033773874384415577, 0.0015439952478456908,...","[1.1193565854990282, 0.4367787381814766, 0.045...","[51983649.084842816, 18467489.390514687, 93113...","[207834810.63940406, 118626816.41525161, 69767...",100.0,
3,ref_1.1,1.10,0.653081,0.163339,3.304638,62.367713,0.285926,0.203817,0.097965,1.973415,2359.262558,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.0037687180020283062, 0.0016886393656873344,...","[1.2712811276325067, 0.35284926376265296, 0.04...","[79229887.83062401, 26717249.389724016, 150764...","[275027165.9278429, 149601425.99259686, 855419...",100.0,
4,best_0.5,0.50,0.047125,0.006460,17.670947,53.388336,0.006372,0.034294,0.004669,1.387612,24474.691629,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.003442748615249063, 0.001745038110784684, 0...","[0.9044947202405491, 0.5579711976004287, 0.043...","[34615254.78513763, 8159922.168290398, 4125302...","[134979135.97680518, 85694095.70253733, 527688...",32.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
5,best_0.65,0.65,0.072464,0.006477,17.765165,57.839677,0.002700,0.063287,0.003866,1.350370,18375.842160,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.0031801424282474396, 0.0017353152775577204,...","[1.0542490372601225, 0.474891897056091, 0.0439...","[34679595.94012743, 10093642.383081028, 466680...","[158912049.23338565, 96055065.44847558, 582259...",32.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
6,best_0.8,0.80,0.173812,0.006947,18.417434,52.564698,0.001690,0.165175,0.005556,1.090753,16272.573955,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.0031275041058928775, 0.001610601155696613, ...","[1.187242027859795, 0.40009593133116683, 0.045...","[48916296.672910966, 13731126.979429752, 67189...","[208171660.72748274, 118317618.22162335, 67788...",32.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
7,best_1.1,1.10,0.604180,0.006227,21.522277,56.192332,0.000795,0.597159,0.004511,1.185415,10791.494064,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.0037283251887850286, 0.0016802970483820155,...","[1.3305024462352648, 0.3195015541079337, 0.046...","[53745890.36225415, 15021292.68224304, 1022339...","[267851166.3543938, 146989706.75764045, 810418...",32.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...


### Compare chi-squares

In [15]:
chi_keys=['label','sigma','chi_1','chi_spec1', 'num_imgs', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','chi_spec3']
df_runs[chi_keys]

,label,sigma,chi_1,chi_spec1,num_imgs,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,chi_spec3
0,ref_0.5,0.50,0.084855,1.027117,100.0,0.057658,0.026169,0.001028,0.030951,0.809446,567.714923,58.889042
1,ref_0.65,0.65,0.000219,0.005281,100.0,0.000076,0.000088,0.000055,0.000021,1.006303,1.631127,49.566061
2,ref_0.8,0.80,0.067837,0.447982,100.0,0.031967,0.029915,0.005955,0.019054,1.404328,431.451944,56.679259
3,ref_1.1,1.10,0.653081,3.304638,100.0,0.163339,0.285926,0.203817,0.097965,1.973415,2359.262558,62.367713
4,best_0.5,0.50,0.047125,17.670947,32.0,0.006460,0.006372,0.034294,0.004669,1.387612,24474.691629,53.388336
5,best_0.65,0.65,0.072464,17.765165,32.0,0.006477,0.002700,0.063287,0.003866,1.350370,18375.842160,57.839677
6,best_0.8,0.80,0.173812,18.417434,32.0,0.006947,0.001690,0.165175,0.005556,1.090753,16272.573955,52.564698
7,best_1.1,1.10,0.604180,21.522277,32.0,0.006227,0.000795,0.597159,0.004511,1.185415,10791.494064,56.192332


## Plot

In [30]:
def f_plot_hist_spec(df,ref_list,plot_type,img_size):

    assert plot_type in ['hist','spec','spec_relative','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))
    
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y=row.spec_val
#             yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            yerr=row.spec_sdev
            x=np.arange(len(y))
            # y=x**2*y; yerr=x**2*yerr ## Plot k^2 P(y)
            plt.fill_between(x, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x, y, marker=marker, linestyle=':',label=label)

        if plot_type=='spec_relative':
            
            y2=row.spec_val
            yerr2=row.spec_sdev
            x2=np.arange(len(y2))
            
            print(row.sigma)
            dict_bkg=bkgnd_dict[str(row.sigma)]


            ### Reference spectrum
            y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r=sqrt(\delta_c2 ^ 2/ c1^2 + \delta_c1^2 * c_2^2 / c_1^4)
            yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row.num_imgs)
    
            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)
            plt.xlim(0,img_size)
            plt.legend()
            plt.title("relative spectrum")
        if plot_type=='grid':
            images=np.load(row.fname)[:,0,:,:,0]
            f_plot_grid(images[:8],cols=4,fig_size=(8,4))

            
    ### Plot input data
    for sig in ref_list: 
        dict_bkg=bkgnd_dict[str(sig)]
        if plot_type=='hist':
            x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
            x=f_invtransform(x)
            plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
            plt.title('Pixel Intensity Histogram')
            plt.xscale('symlog',linthreshx=50)

        if plot_type=='spec':
    #         y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
            y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']

            x=np.arange(len(y))
            # y=x**2*y; yerr=x**2*yerr ## Plot k^2 P(y)
            plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
            plt.title('Spectrum')
            plt.xlim(0,img_size)


        if plot_type=='spec_relative':
            plt.axhline(y=1.0,color='k',linestyle='-.')
            plt.title("relative spectrum")
            plt.xlim(0,img_size)
            plt.ylim(0.5,2)    

        if plot_type in ['hist','spec']:     
            plt.yscale('log')
        plt.legend(bbox_to_anchor=(0.5, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})

    
    plt.show()
# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_runs,dict_val,'spec_relative')
# f_plot_hist_spec(df_best,dict_val,'grid')


In [31]:
df_runs.columns

Index(['label', 'sigma', 'chi_1', 'chi_1a', 'chi_spec1', 'chi_spec3', 'chi_1b',
       'chi_1c', 'chi_2', 'chi_imgvar', 'chi_spec2', 'hist_bin_centers',
       'hist_err', 'hist_val', 'spec_sdev', 'spec_val', 'num_imgs', 'fname'],
      dtype='object')

In [32]:
def f_widget_compare(df,labels_list,bkgnd,ref_list,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,ref_list,plot_type,img_size)

interact_manual(f_widget_compare,df=fixed(df_runs),
                ref_list=SelectMultiple(options=sigma_lst),
                labels_list=SelectMultiple(options=df_runs.label.values), 
                img_size=fixed(img_size),
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','spec_relative','grid']))

interactive(children=(SelectMultiple(description='labels_list', options=('ref_0.5', 'ref_0.65', 'ref_0.8', 're…

<function __main__.f_widget_compare(df, labels_list, bkgnd, ref_list, plot_type)>

In [29]:
col_list=['label','chi_1','chi_spec1','num_imgs','chi_2','chi_spec2']
df_runs[col_list]

,label,chi_1,chi_spec1,num_imgs,chi_2,chi_spec2
0,ref_0.5,0.084855,1.027117,100.0,0.030951,567.714923
1,ref_0.65,0.000219,0.005281,100.0,0.000021,1.631127
2,ref_0.8,0.067837,0.447982,100.0,0.019054,431.451944
3,ref_1.1,0.653081,3.304638,100.0,0.097965,2359.262558
4,best_0.5,0.047125,17.670947,32.0,0.004669,24474.691629
5,best_0.65,0.072464,17.765165,32.0,0.003866,18375.842160
6,best_0.8,0.173812,18.417434,32.0,0.005556,16272.573955
7,best_1.1,0.604180,21.522277,32.0,0.004511,10791.494064


### Test 

In [ ]:
df_runs.fname.values

In [ ]:
# fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210112_191125_cgan_predict_1.1_m2/images/inference_spec_label-1.1_epoch-11_step-25999.npy'
# a1=np.load(fname)
# print(a1.shape)

In [ ]:
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_1.1_train_val.npy'
val_arr=np.load(fname)[:2000][:,0,:,:]
print(val_arr.shape)

In [ ]:
f_pixel_intensity(f_invtransform(a1),normalize=True)
plt.xscale('symlog',linthreshx=50)


# img_lst=[a1]
# f_compare_pixel_intensity(img_lst,label_lst=['a1'],bkgnd_arr=val_arr)


In [ ]:
val_arr=val_arr[:,0,:,:]

In [33]:
np.sqrt(2)*64

90.50966799187809

In [ ]:
img_lst=[a1,val_arr]
f_compare_spectrum(img_lst=img_lst,label_lst=['a1'],bkgnd_arr=val_arr)


### Test histogram

In [ ]:
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_0.5_train_val.npy'
print(fname)
samples=np.load(fname,mmap_mode='r')[-1000:][:,0,:,:]
print(samples.shape)